In [43]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageGrab, ImageTk
import os
import numpy as np
from datetime import datetime
import cv2
from keras.models import load_model

class ScreenshotTool:
    def __init__(self, master):
        self.master = master
        self.master.title("自動截圖工具")

        # Frame 1: 公牌截圖按鈕及顯示區域
        self.board_frame = tk.Frame(self.master)
        self.board_frame.grid(row=0, column=0, padx=10, pady=10)

        self.select_board_points_button = tk.Button(self.board_frame, text="選擇公牌截圖範圍", command=self.start_board_select_points)
        self.select_board_points_button.grid(row=0, column=0, padx=5)

        self.board_image_frame = tk.Frame(self.board_frame)
        self.board_image_frame.grid(row=1, column=0)

        self.board_image_label_list = []
        for i in range(5):
            self.board_image_label = tk.Label(self.board_image_frame)
            self.board_image_label.grid(row=0, column=i, padx=5)  # 橫向排列
            self.board_image_label_list.append(self.board_image_label)

        # Frame 2: 手牌截圖按鈕及顯示區域
        self.hand_frame = tk.Frame(self.master)
        self.hand_frame.grid(row=1, column=0, padx=10, pady=10)

        self.select_hand_points_button = tk.Button(self.hand_frame, text="選擇手牌截圖範圍", command=self.start_hand_select_points)
        self.select_hand_points_button.grid(row=0, column=0, padx=5)

        self.hand_image_frame = tk.Frame(self.hand_frame)
        self.hand_image_frame.grid(row=1, column=0)

        self.hand_image_label_list = []
        for i in range(2):
            self.hand_image_label = tk.Label(self.hand_image_frame)
            self.hand_image_label.grid(row=0, column=i, padx=5)  # 橫向排列
            self.hand_image_label_list.append(self.hand_image_label)


        self.identfier_frame = tk.Frame(self.master)
        self.identfier_frame.grid(row=2, column=0, padx=10, pady=10)

        self.identfier_button = tk.Button(self.identfier_frame, text="辨識牌", command=self.card_identfier)
        self.identfier_button.grid(row=1, column=0, padx=5)

        # 儲存選擇的五個點的座標
        self.board_points = []
        self.hand_points = []
        self.model = load_model('CNN.keras')
        self.categories = [i + j for i in 'A23456789TJQK' for j in "shdc"]

    def card_identfier(self):

        for i in self.board_card_array:
            print(self.categories[self.model.predict(i.reshape(-1, 70, 70, 3)).argmax()])
            

    def start_board_select_points(self):
        # 開始選擇五個點
        self.master.update()
        points = self.get_selected_points()
        self.points = points

        # 顯示截圖
        screenshot = self.get_screenshot()
        self.show_board_screenshot(screenshot)

    def start_hand_select_points(self):
        # 開始選擇五個點
        self.master.update()
        points = self.get_selected_points()
        self.points = points

        # 顯示截圖
        screenshot = self.get_screenshot()
        self.show_hand_screenshot(screenshot)

    def get_selected_points(self):
        # 使用OpenCV擷取點的位置
        ImageGrab.grab(all_screens=True).save("tmp.jpg")
        img = cv2.imread('tmp.jpg')

        points = []
        def get_point(event, x, y, flags, param):
            nonlocal points
            if event == cv2.EVENT_LBUTTONDOWN:
                points.append((x, y))
                cv2.circle(img, (x, y), 5, (255, 0, 0), -1)
                cv2.imshow('SelectPoints', img)

        cv2.namedWindow("SelectPoints", cv2.WINDOW_NORMAL)
        cv2.setWindowProperty("SelectPoints", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('SelectPoints', img)
        cv2.setMouseCallback('SelectPoints', get_point)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return points

    def get_screenshot(self):
        # 根據選擇的五個點擷取截圖
        screenshot_list = []

        for i in self.points:
            screenshot = ImageGrab.grab(bbox=(i[0]-35, i[1]-35, i[0]+35, i[1]+35))
            screenshot_list.append(screenshot)
        return screenshot_list

    #screenshot : img List
    def show_board_screenshot(self, screenshot):
        # 將截圖轉換為Tkinter PhotoImage格式
        self.board_card_array = []
        for i in range(5):
            screenshot_tk = ImageTk.PhotoImage(screenshot[i])
            self.board_card_array.append(np.array(screenshot[i]))

            # 在Label中顯示截圖

            self.board_image_label_list[i].config(image=screenshot_tk)
            self.board_image_label_list[i].image = screenshot_tk

    def show_hand_screenshot(self, screenshot):
        # 將截圖轉換為Tkinter PhotoImage格式
        self.hand_card_array = []
        for i in range(2):
            screenshot_tk = ImageTk.PhotoImage(screenshot[i])
            self.hand_card_array.append(np.array(screenshot[i]))
            # 在Label中顯示截圖

            self.hand_image_label_list[i].config(image=screenshot_tk)
            self.hand_image_label_list[i].image = screenshot_tk

if __name__ == "__main__":
    root = tk.Tk()
    app = ScreenshotTool(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\LALA\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\LALA\AppData\Local\Temp\ipykernel_46860\1524199919.py", line 74, in start_board_select_points
    self.show_board_screenshot(screenshot)
  File "C:\Users\LALA\AppData\Local\Temp\ipykernel_46860\1524199919.py", line 121, in show_board_screenshot
    screenshot_tk = ImageTk.PhotoImage(screenshot[i])
                                       ~~~~~~~~~~^^^
IndexError: list index out of range
